# Imports

In [1]:
conda env list 

# conda environments:
Note: you may need to restart the kernel to use updated packages.
#
pyt3TF2                  C:\Users\Antoine\.conda\envs\pyt3TF2

base                     C:\Users\Antoine\anaconda3
fastai-cpu-v0.7       *  C:\Users\Antoine\anaconda3\envs\fastai-cpu-v0.7
rstudio                  C:\Users\Antoine\anaconda3\envs\rstudio



In [2]:
conda activate fastai-cpu-v0.7


Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd 
from pandas.io.json import json_normalize

In [6]:
#Read OFF full database
database = pd.read_csv(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\data\en.openfoodfacts.org.products.csv', sep='\t',error_bad_lines=False)

In [7]:
database.shape

(1625068, 184)

In [8]:
#Read all robotoff predictions files
pred1 = pd.read_json(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\robotoff_predictions\dump_page1_by_100000.json')
pred2 = pd.read_json(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\robotoff_predictions\dump_page2_by_100000.json')
pred3 = pd.read_json(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\robotoff_predictions\dump_page3_by_100000.json')
pred4 = pd.read_json(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\robotoff_predictions\dump_page4_by_100000.json')
pred5 = pd.read_json(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\robotoff_predictions\dump_page5_by_100000.json')
pred6 = pd.read_json(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\robotoff_predictions\dump_page6_by_100000.json')
pred7 = pd.read_json(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\robotoff_predictions\dump_page7_by_100000.json')
pred8 = pd.read_json(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\robotoff_predictions\dump_page8_by_100000.json')

# Concat and clean predictions files

In [9]:
#Save files into a list to concat them
pred_list = [pred1, pred2, pred3, pred4, pred5, pred6, pred7, pred8]

In [10]:
#Concat files into one df and normalize with json_normalize (for insights column)
raw_predictions = pd.concat(pred_list)
predictions = json_normalize(raw_predictions['insights'])

In [11]:
predictions.head(1)

,id,barcode,type,timestamp,completed_at,annotation,latent,countries,brands,process_after,...,unique_scans_n,reserved_barcode,predictor,username,data.lang,data.model,data.product_name,data.confidence,data.matcher_lang,data.valid
0,454bba15-033a-4f5e-ae9c-3c2a0b847a06,20211189,category,2020-07-31T19:43:38.043274,2021-02-23T18:59:01.437790,-1.0,False,[en:greece],[],None,...,0,True,None,moon-rabbit,fr,matcher,Creme,NaN,NaN,NaN


In [12]:
predictions.shape

(800000, 26)

In [14]:
#Drop unrelevant columns
cols_pred_to_drop = ['id', 'type', 'timestamp', 'latent', 'process_after', 'value','source_image',
'automatic_processing', 'server_domain', 'server_type','unique_scans_n', 'predictor', 'reserved_barcode']
predictions.drop(cols_pred_to_drop, axis=1, inplace=True)

In [15]:
#Add a prefix "robotoff" to the columns (so we can later identify predictions columns into the merged dataframe)
predictions.rename(columns={colname: 'robotoff_' + colname for colname in predictions.columns}, inplace=True)
predictions.rename(columns={'robotoff_barcode' : 'code'}, inplace=True)

In [16]:
predictions.columns

Index(['code', 'robotoff_completed_at', 'robotoff_annotation',
       'robotoff_countries', 'robotoff_brands', 'robotoff_value_tag',
       'robotoff_username', 'robotoff_data.lang', 'robotoff_data.model',
       'robotoff_data.product_name', 'robotoff_data.confidence',
       'robotoff_data.matcher_lang', 'robotoff_data.valid'],
      dtype='object')

# Merge predictions with full database

In [17]:
#Merge predictions with full database
df_with_predictions = pd.merge(database, predictions, on=['code'], how='right')

In [18]:
df_with_predictions.shape

(800001, 196)

In [38]:
len(predictions.code.unique())

615434

In [34]:
predictions.duplicated(subset=['code']).sum()

184566

In [35]:
df_with_predictions.duplicated(subset=['code']).sum()

184567

In [36]:
pd.set_option('display.max_rows', 500)
df_with_predictions.isnull().sum()

code                                               0
url                                           565553
creator                                       565555
created_t                                     565553
created_datetime                              565553
last_modified_t                               565553
last_modified_datetime                        565553
product_name                                  566418
abbreviated_product_name                      800001
generic_name                                  790222
quantity                                      748613
packaging                                     767168
packaging_tags                                767180
packaging_text                                799645
brands                                        677369
brands_tags                                   677378
categories                                    710995
categories_tags                               710996
categories_en                                 

In [20]:
df_with_predictions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 800001 entries, 0 to 800000
Columns: 196 entries, code to robotoff_data.valid
dtypes: float64(125), object(71)
memory usage: 1.2+ GB


# Look different categories columns

In [44]:
df_cats = df_with_predictions[['categories', 'categories_tags', 'categories_en', 'main_category', 'main_category_en', 'robotoff_value_tag']].dropna()

In [45]:
#Stats for differents categories columns
df_cats.describe(include='all')

,categories,categories_tags,categories_en,main_category,main_category_en,robotoff_value_tag
count,89004,89004,89004,89004,89004,89004
unique,19787,12323,12323,7286,7285,3138
top,Snacks,en:snacks,Snacks,en:snacks,Snacks,en:sweetened-beverages
freq,3564,3596,3596,3596,3596,2096


In [46]:
df_cats.duplicated().sum()

52157

In [51]:
#Look categories tags column
pd.set_option('display.max_rows', 500)
df_cats.categories_tags.value_counts().head(10)

en:snacks                                                                                                        3596
en:dairies,en:fermented-foods,en:fermented-milk-products,en:cheeses                                              1600
en:groceries,en:sauces                                                                                           1343
en:snacks,en:sweet-snacks,en:confectioneries                                                                     1313
en:snacks,en:sweet-snacks,en:biscuits-and-cakes,en:biscuits                                                      1215
en:beverages                                                                                                      987
en:plant-based-foods-and-beverages,en:beverages,en:plant-based-beverages                                          906
en:beverages,en:carbonated-drinks,en:sodas                                                                        841
en:snacks,en:sweet-snacks,en:confectioneries,en:chocolat

In [52]:
#Look main category column
df_cats.main_category.value_counts().head(20)

en:snacks                        3596
en:cheeses                       1609
en:sauces                        1390
en:confectioneries               1315
en:biscuits                      1242
en:sweetened-beverages           1176
en:beverages                      987
en:plant-based-beverages          907
en:sodas                          881
en:salted-snacks                  829
en:chocolate-candies              827
en:cereals-and-their-products     816
en:condiments                     737
en:frozen-desserts                721
en:breads                         687
en:extra-virgin-olive-oils        680
en:prepared-meats                 627
en:yogurts                        622
en:candies                        496
en:waters                         463
Name: main_category, dtype: int64

In [53]:
#Look robotoff values predictions column
df_with_predictions['robotoff_value_tag'].value_counts().head(20)

en:cheeses                13145
en:biscuits               13139
en:breads                 12593
en:sauces                 10565
en:sweetened-beverages    10212
en:chocolates              9836
en:yogurts                 8147
en:confectioneries         7601
en:candies                 7532
en:condiments              6496
en:breakfast-cereals       6411
en:ice-creams              6291
en:teas                    5984
en:dark-chocolates         5519
en:chickens                5381
en:coffees                 5238
en:cookies                 5051
en:pastas                  4875
en:sausages                4803
en:fruit-juices            4791
Name: robotoff_value_tag, dtype: int64

# Export dataframes to csv files

In [50]:
#Full merged dataframe
df_with_predictions.to_csv(r'C:\Users\Antoine\Coding Bootcamp\machine learning\Open Food Facts\data\merged_predictions_off_full.csv', index=False, header=True)